# Results of my local various tryouts 

In this notebook I present results obtained on training different networks with several types of dataflow.

## TLDR

- 


## More details

### Imbalanced data

As we see from the tutorial notebook classes are not equally represented, there are even classes with only 4 images.  It will be very difficult to train a network to predict correctly such classes. For more info on data see [this notebook](notebooks/data_visualization.ipynb)

#### Class weights 

A basic solution that can be done is to add `class_weights` to keras. I did that in the following manner: 
- compute class count from training data generator
- define weights as 
```
class_weights[class_index] = np.log(1.0 + (max_count / class_weights[class_index])**2)
```

#### Random over/under-sampling

There is an option to oversample training dataset using non-trivial data augmentations. I used [sklearn-imbalanced](https://github.com/scikit-learn-contrib/imbalanced-learn) for random over/under-samplings. However, application of basic classes oversampling creates too much data. The most represented class has 4875 images, thus random oversampling can generate about `4875 * 403 = 1964625` images. I tested such approach in the very beginning.

Another approach is to undersample most common classes to a certain number and than oversample rare classes. Such approach gives less images to train on comparing to the previous approach. The difficulty here is to undersample correctly most common classes. A solution can be to cluster by color images from a single class and pick a balanced number of each cluster representatives to form. Unfortunately, such approach requires to handle data directly, without using provided `gen_builder` and this is forbidden by organizers. The code of this approach can be found [here](notebooks/balanced_dataflow.ipynb). There is still a solution to choose images randomly in the undersampling phase. Here is a code using `sklearn-imbalanced` to create generators with under+oversampled balanced data:
```
    # Under+Oversample training data:
    # - undersample randomly images that count is larger a threshold
    # - oversample randomly all images

    undersampling_threshold = 350

    class_counts = np.zeros((403, ), dtype=np.int)
    for class_index in self.y_array[train_indices]:
        class_counts[class_index] += 1

    classes_to_undersample = np.where(class_counts > undersampling_threshold)[0]

    train_indices_to_undersample = [index for index in train_indices if self.y_array[index] in classes_to_undersample]
    train_indices_to_oversample = [index for index in train_indices if
                                   self.y_array[index] not in classes_to_undersample]

    rs = RandomUnderSampler()

    train_indices_undersampled, new_y_array = rs.fit_sample(np.array(train_indices_to_undersample)[:, None],
                                                            self.y_array[train_indices_to_undersample])
    rs = RandomOverSampler()
    new_train_indices = np.concatenate((train_indices_undersampled, np.array(train_indices_to_oversample)[:, None]))
    new_y_array = np.concatenate((new_y_array, self.y_array[train_indices_to_oversample]))

    new_train_indices, _ = rs.fit_sample(new_train_indices, new_y_array)
    new_train_indices = new_train_indices.ravel()
    gen_train = self._get_generator(indices=new_train_indices, batch_size=batch_size)
    nb_train = len(new_train_indices)

```
However, my tests did not show a gain using such approach.

#### Multiple stage training approach

Intensively using the first approach of `class_weights` I found that 


### Model architectures

I tried to finetune 3 networks available in keras : VGG16, Inception-ResNetV2, DenseNet



### Hardware

- Nvidia 1080 ti
- i7 
- 32Gb RAM
- SSD